In [1]:
import cv2

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
print(frame.shape)

from __future__ import print_function
import sys
sys.path.insert(0, 'src')
import transform, numpy as np, vgg, pdb, os
import scipy.misc
import tensorflow as tf
from utils import save_img, get_img, exists, list_files
from argparse import ArgumentParser
from collections import defaultdict
import time
import json
import subprocess
import numpy

batch_size = 1
g = tf.Graph()
soft_config = tf.ConfigProto(allow_soft_placement=True)
soft_config.gpu_options.allow_growth = True

checkpoint_dir = "checkpoints"
#checkpoints/la_muse.ckpt
#checkpoints/rain_princess.ckpt
#checkpoints/scream.ckpt
#checkpoints/udnie.ckpt
#checkpoints/wave.ckpt
#checkpoints/wreck.ckpt

with g.as_default(), g.device('/gpu:0'), tf.Session(config=soft_config) as sess:
    batch_shape = (batch_size,) +  frame.shape
    img_placeholder = tf.placeholder(tf.float32, shape=batch_shape, name='img_placeholder')

    preds = transform.net(img_placeholder)
    saver = tf.train.Saver()
    if os.path.isdir(checkpoint_dir):
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
        else:
            raise Exception("No checkpoint found...")
    else:
        saver.restore(sess, checkpoint_dir)
    
    X = np.zeros(batch_shape, dtype=np.float32)
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Our operations on the frame come here
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        X[0] = frame
        _preds = sess.run(preds, feed_dict={img_placeholder:X})
        print(_preds)
        image = np.clip(_preds[0], 0, 255).astype(np.uint8)
        # Display the resulting frame
        cv2.imshow('frame',image)

        if cv2.waitKey(30) & 0xFF == ord('q'): #waitKey의 value의 값이 너무 높으면 너무 자주 key값 확인을 해서 느려짐, 프레임에 맞게 설정
            break


# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

(480, 640, 3)
[[[[  59.85719299   81.22892761  101.82919312]
   [  62.06547546   86.53909302  100.43855286]
   [  56.8952179    76.15692139   93.95701599]
   ..., 
   [  53.55329895   77.65263367   92.27194214]
   [  53.57387543   71.7490921    93.91362762]
   [  54.27913666   74.60327148   98.39305115]]

  [[  64.22590637   90.5226593   109.34170532]
   [  67.95671082   95.72719574  108.42987823]
   [  60.34902191   77.57757568   98.73355103]
   ..., 
   [  61.95956421   83.09150696   91.75202942]
   [  57.61194611   75.88444519   92.48175049]
   [  63.67958069   83.31230927   98.90909576]]

  [[  68.8941803    92.33362579  109.87113953]
   [  76.21559143   99.23529816  108.53793335]
   [  63.60032272   74.89501953   92.77114105]
   ..., 
   [  56.55715942   65.30109406   72.9930191 ]
   [  69.302948     80.41470337   87.50921631]
   [  69.22761536   83.27944183   94.06761932]]

  ..., 
  [[  57.24939728   77.20405579   92.85964966]
   [  60.64076996   81.29598999   84.50987244]
   [ 